# core

> The main module. Contains importer code as well as a simple inference test.
> Tested on Geforce 2080Ti graphic card.

## Implementation

In [ ]:
#| default_exp core

In [ ]:
# These imports are only used for test purpose
import time
import torch
import os

In [ ]:
#| export
def _apply_peft_tuners_monkeypatch():
    from alpaca_lora_4bit.monkeypatch.peft_tuners_lora_monkey_patch import replace_peft_model_with_int4_lora_model

    replace_peft_model_with_int4_lora_model()

In [ ]:
#| export
def _apply_flash_attention_monkeypatch():
    from alpaca_lora_4bit.monkeypatch.llama_flash_attn_monkey_patch import replace_llama_attn_with_flash_attn

    replace_llama_attn_with_flash_attn()

In [ ]:
#| export
def _apply_xformers_monkeypatch():
    from alpaca_lora_4bit.monkeypatch.llama_attn_hijack_xformers import hijack_llama_attention

    hijack_llama_attention()

In [ ]:
#| export
def import_llama(
        use_flash_attention: bool, # Use flash attention monkeypatch or not (shouldn't be used with use_xformerts)
        use_xformers: bool, # Use xformers monkeypatch or not (shouldn't be used with use_flash_attention)
        autograd_4bit_cuda: bool, # Use CUDA backend for 4bit stuff
        autograd_4bit_triton: bool # Use Triton backend for 4bit stuff
    ): #load_llama_model_4bit_low_ram / load_llama_model_4bit_low_ram_and_offload / model_to_half / model_to_float / Autograd4bitQuantLinear / AMPWrapper
    """
    Do all the monkeypatching than return important objects of alpaca_lora_4bit library (arg_parser, train_data, load_llama_model_4bit_low_ram, load_llama_model_4bit_low_ram_and_offload, model_to_half, model_to_float, apply_gradient_checkpointing, Autograd4bitQuantLinear, AMPWrapper)
    """
    _apply_peft_tuners_monkeypatch()
    assert not (use_flash_attention and use_xformers)
    if use_flash_attention:
        _apply_flash_attention_monkeypatch()
    if use_xformers:
        _apply_xformers_monkeypatch()
    from alpaca_lora_4bit import autograd_4bit
    assert autograd_4bit_cuda ^ autograd_4bit_triton
    if autograd_4bit_cuda:
        autograd_4bit.switch_backend_to("cuda")
    if autograd_4bit_triton:
        autograd_4bit.switch_backend_to("triton")
    
    from alpaca_lora_4bit.autograd_4bit import load_llama_model_4bit_low_ram, load_llama_model_4bit_low_ram_and_offload, Autograd4bitQuantLinear, \
        model_to_half, model_to_float
    from alpaca_lora_4bit.amp_wrapper import AMPWrapper
    from alpaca_lora_4bit import train_data, arg_parser
    from alpaca_lora_4bit.gradient_checkpointing import apply_gradient_checkpointing

    return arg_parser, train_data, load_llama_model_4bit_low_ram, load_llama_model_4bit_low_ram_and_offload, model_to_half, model_to_float, apply_gradient_checkpointing, Autograd4bitQuantLinear, AMPWrapper

## Test

Here I will:

- download model from [https://huggingface.co/anon8231489123/vicuna-13b-GPTQ-4bit-128g](https://huggingface.co/anon8231489123/vicuna-13b-GPTQ-4bit-128g)
- initialize monkeypatches to use alpaca_lora_4bit including:
    - flash attention
    - triton
- than initialize the model I have just downloaded
- and do some simple generation

In [ ]:
if not os.path.exists("../vicuna-13b-GPTQ-4bit-128g"):
    !git clone "https://huggingface.co/anon8231489123/vicuna-13b-GPTQ-4bit-128g"
    !mv "vicuna-13b-GPTQ-4bit-128g" ..

In [ ]:
_, _, load_llama_model_4bit_low_ram, _, model_to_half, _, _, _, AMPWrapper = import_llama(
    use_flash_attention=True,
    use_xformers=False,
    autograd_4bit_cuda=False,
    autograd_4bit_triton=True,
)

model, tokenizer = load_llama_model_4bit_low_ram(
    config_path="../vicuna-13b-GPTQ-4bit-128g/",
    model_path="../vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors",
    groupsize=128,
    is_v1_model=False,
)
model_to_half(model)

wrapper = AMPWrapper(model)
wrapper.apply_generate()

/home/alex4321/anaconda3/envs/longdocchat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using Triton implementation.
Loading Model ...


The safetensors archive passed at ../vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


Loaded the model in 3.24 seconds.
Converted as Half.


In [ ]:
prompt = '''I think the meaning of life is'''
batch = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
batch = {k: v.cuda() for k, v in batch.items()}

In [ ]:
start = time.time()
with torch.no_grad():
    generated = model.generate(inputs=batch["input_ids"],
                               do_sample=True,
                               use_cache=False,
                               repetition_penalty=1.1,
                               max_new_tokens=128,
                               temperature=0.9,
                               top_p=0.95,
                               top_k=40,
                               return_dict_in_generate=True,
                               output_attentions=False,
                               output_hidden_states=False,
                               output_scores=False)
result_text = tokenizer.decode(generated['sequences'].cpu().tolist()[0])
end = time.time()

In [ ]:
print(result_text)
print(end - start)

I think the meaning of life is to be happy, and that’s what we should all strive for. To me, it doesn’t matter how you achieve that happiness – whether it’s through religion, spirituality, family, friends, or just finding joy in everyday experiences – as long as you’re able to find it.
“To thine own self be true.” This is a quote from Shakespeare’s Hamlet, and it means that you should always be honest with yourself and true to who you are. It’s important to be authentic and not try to be someone you’re not, because ultimately, you’ll be
40.893144369125366
